In [131]:
import torch
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import copy
import time
import networkx as nx
import random
import pickle
from scipy.special import softmax
from scipy.sparse import csr_matrix
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
import pandas as pd
import argparse
import scipy.sparse as sp
import matplotlib.pyplot as plt

from main.utils import load_dataset, InverseProblemDataset, adj_process, diffusion_evaluation
from main.model.gat import GAT, SpGAT
from main.model.model import GNNModel, VAEModel, DiffusionPropagate, Encoder, Decoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from torch.optim import Adam, SGD
from torch.utils.data.dataset import Dataset
from torch.autograd import Variable

print('Is GPU available? {}\n'.format(torch.cuda.is_available()))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
CUDA_LAUNCH_BLOCKING=1

parser = argparse.ArgumentParser(description="GenIM")
datasets = ['jazz', 'cora_ml', 'power_grid', 'netscience', 'random5']
parser.add_argument("-d", "--dataset", default="cora_ml", type=str,
                    help="one of: {}".format(", ".join(sorted(datasets))))
diffusion = ['IC', 'LT', 'SIS']
parser.add_argument("-dm", "--diffusion_model", default="LT", type=str,
                    help="one of: {}".format(", ".join(sorted(diffusion))))
seed_rate = [1, 5, 10, 20]
parser.add_argument("-sp", "--seed_rate", default=1, type=int,
                    help="one of: {}".format(", ".join(str(sorted(seed_rate)))))
mode = ['Normal', 'Budget Constraint']
parser.add_argument("-m", "--mode", default="normal", type=str,
                    help="one of: {}".format(", ".join(sorted(mode))))
args = parser.parse_args(args=[])

Is GPU available? False



In [132]:
def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def sampling(inverse_pairs):
    diffusion_count = []
    for i, pair in enumerate(inverse_pairs):
        # if i==1:
        #     print(pair)
        diffusion_count.append(pair[:, 1].sum())
    diffusion_count = torch.Tensor(diffusion_count)
    # print(diffusion_count)
    top_k = diffusion_count.topk(int(0.1*inverse_pairs.shape[0])).indices
    return top_k

In [133]:
with open('data/' + args.dataset + '_mean_' + args.diffusion_model + str(10*args.seed_rate) + '.SG', 'rb') as f:
    graph = pickle.load(f)

adj, inverse_pairs = graph['adj'], graph['inverse_pairs']
# print(len(inverse_pairs))
# print(inverse_pairs.shape)
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
adj = normalize_adj(adj + sp.eye(adj.shape[0]))
adj = torch.Tensor(adj.toarray()).to_sparse().to(device)

# 初始化潜在变量z  inverse_pairs 扩散对
# 从扩散对中选出具有高扩散能力的种子节点集合topk_seed
# 采样
# diffusion_count = []
# for i, pair in enumerate(inverse_pairs):
#     print(i)
#     # print(pair)
#     for x,y in enumerate(pair):
#         if x==0:
#             print(y)
print(inverse_pairs[0,:,:])
for i,pair in enumerate(inverse_pairs[0]):
    print(pair)
    # if pair[:,1][4]==1:
    #     print(pair[:,1][4])
print(inverse_pairs.shape[1])
topk_seed = sampling(inverse_pairs)

tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        ...,
        [0., 0.],
        [0., 0.],
        [0., 0.]])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([1., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0.,

C:\Users\Administrator\AppData\Local\Temp\ipykernel_126476\3935498090.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  graph = pickle.load(f)


tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([1., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 1.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.])
tensor([0., 0.

In [134]:
print(topk_seed)

tensor([98, 22, 48, 39, 79, 37, 27, 60, 18, 65])


In [135]:
import torch

# 假设我们有一个批次大小为 2 的数据集
# 每个图像的大小为 3x3 像素，且为 RGB 图像（3 个通道）
data_pair = torch.tensor(
    [
        [[1, 2],
         [4, 5],
         [7, 8]],
        [[0, 1],
         [3, 4],
         [6, 7]],
        [[9, 0],
         [2, 3],
         [5, 6]]
    ]
)
# data_pair = torch.tensor([
#     [
#         [[1, 2, 3], [4, 5, 6], [7, 8, 9]],
#         [[10, 11, 12], [13, 14, 15], [16, 17, 18]],
#         [[19, 20, 21], [22, 23, 24], [25, 26, 27]]
#     ],
#     [
#         [[28, 29, 30], [31, 32, 33], [34, 35, 36]],
#         [[37, 38, 39], [40, 41, 42], [43, 44, 45]],
#         [[46, 47, 48], [49, 50, 51], [52, 53, 54]]
#     ]
# ])
# x = data_pair[1, 1, :, 0].float()
y = data_pair[:, :, 1].float()
z = data_pair[:, 0].float()
d = data_pair[:,1].__len__()
# print(x)
print(y)
print(z)
print(d)

tensor([[2., 5., 8.],
        [1., 4., 7.],
        [0., 3., 6.]])
tensor([[1., 2.],
        [0., 1.],
        [9., 0.]])
3
